In [124]:
# Instalar DVC
!pip install dvc

In [125]:
# Para llevar el registro
!pip install comet_ml
import comet_ml
comet_ml.init(project_name = 'Tarea_4_CatsDogs')

In [126]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2,l1

In [127]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Prueba 04')

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-4-catsdogs/79007b6c2e8c4cc192634974b9317f94
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     batch_accuracy [20]         : (0.4970967769622803, 0.5410470962524414)
COMET INFO:     batch_loss [20]             : (1.5741530656814575, 2.0059123039245605)
COMET INFO:     validate_batch_accuracy [5] : (0.5490244030952454, 0.5799999833106995)
COMET INFO:     validate_batch_loss [5]     : (1.2155985832214355, 1.2222415208816528)
COMET INFO:   Others:
COMET INFO:     Name             : Prueba 03
COMET INFO:     trainable_params : 21661
COMET INFO:   Parameters:
COMET INFO:     Adam_amsgrad                 : F

In [128]:
# Dimensiones de las imágenes
image_w = 150
image_h = 150
input_shape = (image_w, image_h, 3)

# Directorio
train_dir='/kaggle/input/dogscats/train'
test_dir='/kaggle/input/dogscats/test'

In [129]:
# Parámetros
parameters = {
    'num_class' : 2,
    'epochs' : 40,
    'batch_size' : 100,
    'learning_rate' : 0.0001,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'Adam',
    'num_train' : 20000,
    'num_test' : 5000,
}
experiment.log_parameters(parameters)

In [130]:
# Solo toma el entero
epoch_steps = parameters['num_train'] // parameters['batch_size']
test_steps = parameters['num_test'] // parameters['batch_size']

In [131]:
# Se preparan las imágenes
gentrain = ImageDataGenerator(rescale = 1./255.)                                   # Rescala entre 0 y 1 cada canal
train = gentrain.flow_from_directory(train_dir,                                    # Directorio donde se leen las imágenes
                                   batch_size = parameters['batch_size'],          
                                   target_size = (image_w,image_h),                # Redimensiona las imágenes
                                   class_mode = 'binary')                          # Tipo de clasificación

gentest = ImageDataGenerator(rescale = 1./255.)
test = gentest.flow_from_directory(test_dir,
                                 target_size=(image_w,image_h),
                                 batch_size=parameters['batch_size'],
                                 class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [132]:
# Arquitectura
model = Sequential()

model.add(Conv2D(10,(3,3), input_shape = (input_shape)))    # 10 mapas de características | ventana de atención de 3x3
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(20, (3,3), kernel_regularizer = l2(0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(40, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(20, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(10, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

          
model.add(Flatten())
          
model.add(Dense(50, kernel_regularizer = l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(20, kernel_regularizer = l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.05))          

model.add(Dense(10, kernel_regularizer = l1(0.01)))
model.add(Activation('relu'))
# model.add(Dropout(0.2))
          
model.add(Dense(1))
model.add(Activation('sigmoid'))
          
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_57 (Conv2D)          (None, 148, 148, 10)      280       
                                                                 
 activation_93 (Activation)  (None, 148, 148, 10)      0         
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 74, 74, 10)       0         
 g2D)                                                            
                                                                 
 conv2d_58 (Conv2D)          (None, 72, 72, 20)        1820      
                                                                 
 activation_94 (Activation)  (None, 72, 72, 20)        0         
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 36, 36, 20)       0         
 g2D)                                                

In [133]:
# Checkpoint para el mejor modelo (val_loss)
filepath = 'best_val_loss.hdf5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min'
)

In [134]:
# Compilación y entrenamiento
model.compile(
    loss = parameters['loss'],
    optimizer = Adam(learning_rate = 0.0005),
    metrics=['accuracy']
)
model.fit(
    train,
    batch_size = parameters['batch_size'],
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = test,
    steps_per_epoch = epoch_steps,
    validation_steps = test_steps,
    callbacks = [checkpoint]
)


Epoch 1/40
200/200 [==============================] - ETA: 0s - loss: 1.5642 - accuracy: 0.5336
Epoch 1: val_loss improved from inf to 1.22543, saving model to best_val_loss.hdf5
200/200 [==============================] - 69s 319ms/step - loss: 1.5642 - accuracy: 0.5336 - val_loss: 1.2254 - val_accuracy: 0.5218
Epoch 2/40
200/200 [==============================] - ETA: 0s - loss: 1.0467 - accuracy: 0.5637
Epoch 2: val_loss improved from 1.22543 to 0.89902, saving model to best_val_loss.hdf5
200/200 [==============================] - 47s 236ms/step - loss: 1.0467 - accuracy: 0.5637 - val_loss: 0.8990 - val_accuracy: 0.5994
Epoch 3/40
200/200 [==============================] - ETA: 0s - loss: 0.8103 - accuracy: 0.6260
Epoch 3: val_loss improved from 0.89902 to 0.73614, saving model to best_val_loss.hdf5
200/200 [==============================] - 48s 238ms/step - loss: 0.8103 - accuracy: 0.6260 - val_loss: 0.7361 - val_accuracy: 0.6376
Epoch 4/40
200/200 [==============================] -

In [135]:
experiment.log_model("CatsDogs", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-4-catsdogs/024e9593c83c45d6b0281613b1a78245
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [40]                 : (0.5335500240325928, 0.8647500276565552)
COMET INFO:     batch_accuracy [800]          : (0.49272727966308594, 0.9100000262260437)
COMET INFO:     batch_loss [800]              : (0.3180583119392395, 2.03010630607605)
COMET INFO:     epoch_duration [40]           : (45.74862812799984, 68.4136255960002)
COMET INFO:     loss [40]                     : (0.38472747802734375, 1.564191460609436)
COMET INFO:     val_accuracy [40]             : (0.5217999815940857, 0.8496000170707703)
